<a href="https://colab.research.google.com/github/Nilanshrajput/NER_SyferText/blob/master/lSTM_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!git clone https://github.com/synalp/NER.git

Cloning into 'NER'...
remote: Enumerating objects: 3148, done.
remote: Total 3148 (delta 0), reused 0 (delta 0), pack-reused 3148
Receiving objects: 100% (3148/3148), 281.51 MiB | 30.58 MiB/s, done.
Resolving deltas: 100% (2066/2066), done.
Checking out files: 100% (189/189), done.


In [0]:

import torch
from torchtext import data
from torchtext.datasets import SequenceTaggingDataset, CoNLL2000Chunking
from torchtext.vocab import Vectors, GloVe, CharNGram

import numpy as np
import random
import logging
logger = logging.getLogger(__name__)


def conll2003_dataset(tag_type, batch_size, root='/content/NER/corpus/CoNLL-2003', 
                          train_file='eng.train', 
                          validation_file='eng.testa',
                          test_file='eng.testb',
                          convert_digits=True):
    """
    conll2003: Conll 2003 (Parser only. You must place the files)
    Extract Conll2003 dataset using torchtext. Applies GloVe 6B.200d and Char N-gram
    pretrained vectors. Also sets up per word character Field
    Parameters:
        tag_type: Type of tag to pick as task [pos, chunk, ner]
        batch_size: Batch size to return from iterator
        root: Dataset root directory
        train_file: Train filename
        validation_file: Validation filename
        test_file: Test filename
        convert_digits: If True will convert numbers to single 0's
    Returns:
        A dict containing:
            task: 'conll2003.' + tag_type
            iters: (train iter, validation iter, test iter)
            vocabs: (Inputs word vocabulary, Inputs character vocabulary, 
                    Tag vocabulary )
    """
    
    # Setup fields with batch dimension first
    inputs_word = data.Field(init_token="<bos>", eos_token="<eos>",fix_length=100, batch_first=True, lower=True)

    inputs_char_nesting = data.Field(tokenize=list, init_token="<bos>", eos_token="<eos>", 
                                    batch_first=True)

    inputs_char = data.NestedField(inputs_char_nesting, 
                                    init_token="<bos>", eos_token="<eos>")
                        

    labels = data.Field(init_token="<bos>", eos_token="<eos>", fix_length=50, batch_first=True)

    fields = ([(('inputs_word', 'inputs_char'), (inputs_word, inputs_char))] + 
                [('labels', labels) if label == tag_type else (None, None) 
                for label in ['pos', 'chunk', 'ner']])

    # Load the data
    train, val, test = SequenceTaggingDataset.splits(
                                path=root, 
                                train=train_file, 
                                validation=validation_file, 
                                test=test_file,
                                separator=' ',
                                fields=tuple(fields))


    
    # Build vocab
    inputs_char.build_vocab(train.inputs_char, val.inputs_char, test.inputs_char)
    inputs_word.build_vocab(train.inputs_word, val.inputs_word, test.inputs_word, max_size=50000,
                        vectors=[GloVe(name='6B', dim='300'), CharNGram()])
    
    labels.build_vocab(train.labels)
  

    # Get iterators
    train_iter, val_iter, test_iter = data.BucketIterator.splits(
                            (train, val, test), batch_size=batch_size, 
                            device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))
    train_iter.repeat = False
    
    return {
        'task': 'conll2003.%s'%tag_type,
        'iters': (train_iter, val_iter, test_iter), 
        'vocabs': (inputs_word.vocab, inputs_char.vocab, labels.vocab) 
        }
    


In [0]:
dic = conll2003_dataset('ner', batch_size = 64)

.vector_cache/glove.6B.zip: 862MB [06:27, 2.23MB/s]                           
100%|█████████▉| 399564/400000 [00:51<00:00, 7591.87it/s]
.vector_cache/jmt_pre-trained_embeddings.tar.gz: 0.00B [00:00, ?B/s]
.vector_cache/jmt_pre-trained_embeddings.tar.gz:   0%|          | 8.19k/956M [00:03<119:32:01, 2.22kB/s]
.vector_cache/jmt_pre-trained_embeddings.tar.gz:   0%|          | 41.0k/956M [00:03<83:58:37, 3.16kB/s] 
.vector_cache/jmt_pre-trained_embeddings.tar.gz:   0%|          | 106k/956M [00:03<58:56:00, 4.50kB/s] 
.vector_cache/jmt_pre-trained_embeddings.tar.gz:   0%|          | 172k/956M [00:04<41:24:12, 6.41kB/s]
.vector_cache/jmt_pre-trained_embeddings.tar.gz:   0%|          | 369k/956M [00:04<29:01:39, 9.14kB/s]
.vector_cache/jmt_pre-trained_embeddings.tar.gz:   0%|          | 762k/956M [00:04<20:20:11, 13.0kB/s]
.vector_cache/jmt_pre-trained_embeddings.tar.gz:   0%|          | 1.55M/956M [00:04<14:14:11, 18.6kB/s]
.vector_cache/jmt_pre-trained_embeddings.tar.gz:   0%|          | 3

In [0]:
while 1:
  continue

In [0]:
a,b,c = dic['iters']

In [0]:
vocab,_,_ = dic['vocabs']

In [52]:
vsz = len(vocab)
print(vsz)

26874


In [56]:
t
for t in enumerate(a):
  print(t)
  print(len(t))
  t=t[1]
  print(t[1])
  

(0, 
[torchtext.data.batch.Batch of size 1]
	[.labels]:[torch.LongTensor of size 1x50]
	[.inputs_word]:[torch.LongTensor of size 1x100]
	[.inputs_char]:[torch.LongTensor of size 1x9x12])
2


TypeError: ignored

In [57]:
t


[torchtext.data.batch.Batch of size 1]
	[.labels]:[torch.LongTensor of size 1x50]
	[.inputs_word]:[torch.LongTensor of size 1x100]
	[.inputs_char]:[torch.LongTensor of size 1x9x12]

In [58]:
t.inputs_word.shape

torch.Size([1, 100])

In [0]:
import torch.nn as nn
import torch.nn.functional as F

class LSTM_Tagger(nn.Module):
    def __init__(self, params):
        super(LSTM_Tagger, self).__init__()

        #maps each token to an embedding_dim vector
        self.embedding = nn.Embedding(params['vocab_size'], params['embedding_dim'])

        #the LSTM takens embedded sentence
        self.lstm = nn.LSTM(params['embedding_dim'], params['lstm_hidden_dim'], batch_first=True)

        #fc layer transforms the output to give the final output layer
        self.fc = nn.Linear(params['lstm_hidden_dim'], params['num_tags'])

    def forward(self, s):
        #apply the embedding layer that maps each token to its embedding
        s = self.embedding(s)   # dim: batch_size x batch_max_len x embedding_dim

        #run the LSTM along the sentences of length batch_max_len
        s, _ = self.lstm(s)     # dim: batch_size x batch_max_len x lstm_hidden_dim                

        #reshape the Variable so that each row contains one token
        s = s.view(-1, s.shape[2])  # dim: batch_size*batch_max_len x lstm_hidden_dim

        #apply the fully connected layer and obtain the output for each token
        s = self.fc(s)          # dim: batch_size*batch_max_len x num_tags

        return F.log_softmax(s, dim=1)   # dim: batch_size*batch_max_len x num_tags


In [0]:
embedding = nn.Embedding(vsz, 300)

In [0]:
print(embedding(t.inputs_word).shape)

torch.Size([1, 34, 300])


In [0]:
def loss_fn(outputs, labels):
    #reshape labels to give a flat vector of length batch_size*seq_len
    labels = labels.view(-1)  

    #mask out 'PAD' tokens
    mask = (labels >= 0).float()

    #the number of tokens is the sum of elements in mask
    num_tokens = int(torch.sum(mask).data[0])

    #pick the values corresponding to labels and multiply by mask
    outputs = outputs[range(outputs.shape[0]), labels]*mask

    #cross entropy loss for all non 'PAD' tokens
    return -torch.sum(outputs)/num_tokens

def clip_gradient(model, clip_value):
    params = list(filter(lambda p: p.grad is not None, model.parameters()))
    for p in params:
        p.grad.data.clamp_(-clip_value, clip_value)
    
def train_model(model, train_iter, epoch):
    total_epoch_loss = 0
    total_epoch_acc = 0
    model.cuda()
    optim = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()))
    steps = 0
    model.train()
    for idx, batch in enumerate(train_iter):
        text = batch.inputs_word
        print(text)
        print(text.shape)
        print(text[1])
        break
        target = batch.labels
        target = torch.autograd.Variable(target).long()
        if torch.cuda.is_available():
            text = text.cuda()
            target = target.cuda()
        if (text.size()[0] is not 32):# One of the batch returned by BucketIterator has length different than 32.
            continue
        optim.zero_grad()
        prediction = model(text)
        loss = loss_fn(prediction, target)
        num_corrects = (torch.max(prediction, 1)[1].view(target.size()).data == target.data).float().sum()
        acc = 100.0 * num_corrects/len(batch)
        loss.backward()
        clip_gradient(model, 1e-1)
        optim.step()
        steps += 1
        
        if steps % 100 == 0:
            print (f'Epoch: {epoch+1}, Idx: {idx+1}, Training Loss: {loss.item():.4f}, Training Accuracy: {acc.item(): .2f}%')
        
        total_epoch_loss += loss.item()
        total_epoch_acc += acc.item()
        
    return total_epoch_loss/len(train_iter), total_epoch_acc/len(train_iter)

def eval_model(model, val_iter):
    total_epoch_loss = 0
    total_epoch_acc = 0
    model.eval()
    with torch.no_grad():
        for idx, batch in enumerate(val_iter):
            text = batch.text[0]
            if (text.size()[0] is not 32):
                continue
            target = batch.label
            target = torch.autograd.Variable(target).long()
            if torch.cuda.is_available():
                text = text.cuda()
                target = target.cuda()
            prediction = model(text)
            loss = loss_fn(prediction, target)
            num_corrects = (torch.max(prediction, 1)[1].view(target.size()).data == target.data).sum()
            acc = 100.0 * num_corrects/len(batch)
            total_epoch_loss += loss.item()
            total_epoch_acc += acc.item()

    return total_epoch_loss/len(val_iter), total_epoch_acc/len(val_iter)
	



In [2]:
learning_rate = 2e-5
batch_size = 64

lstm_hidden_dim = 256
num_tags = 17
embedding_length = 300
train_iter,valid_iter,test_iter = dic['iters']
vsz = len(dic['vocabs'])
params={'vocab_size':vsz, 'embedding_dim':embedding_length,'lstm_hidden_dim': lstm_hidden_dim, 'num_tags':num_tags}
model = LSTM_Tagger(params)
for epoch in range(10):
    train_loss, train_acc = train_model(model, train_iter, epoch)
    val_loss, val_acc = eval_model(model, valid_iter)
    
    print(f'Epoch: {epoch+1:02}, Train Loss: {train_loss:.3f}, Train Acc: {train_acc:.2f}%, Val. Loss: {val_loss:3f}, Val. Acc: {val_acc:.2f}%')
    
test_loss, test_acc = eval_model(model, test_iter)
print(f'Test Loss: {test_loss:.3f}, Test Acc: {test_acc:.2f}%')

NameError: ignored

In [7]:
params={'vocab_size':5, 'embedding_dim':5}
params.vocab_size

AttributeError: ignored